# Projeto Final Big Data - Reviews Steam 2021

Projeto para a disciplina de Big Data sobre reviews na Steam de 2021, realizando uma análise exploratória e uma predição baseada em machine learning para prever se um jogo receberia uma review positiva ou negativa baseado em alguns parâmetros.

**Dataset**: https://www.kaggle.com/datasets/najzeko/steam-reviews-2021

**Grupo**: Guilherme Lunetta, Rafael Monteiro e João Vitor Magalhães

In [1]:
import dask
from dask.distributed import Client
import dask.dataframe as dd
import dask.multiprocessing
import plotly.express as px

### Cliente DASK

In [ ]:
# Rodar apenas se quiser fechar a conexão

client.close()

In [2]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53977,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:54004,Total threads: 2
Dashboard: http://127.0.0.1:54007/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:53980,


### Abrindo o dataset e dando os primeiros passos

In [4]:
# Foi necessário colocar dtype em todas as colunas pois o dask estava inferindo muito mal o tipo de cada coluna,
# dificultando muito a análise. Também foi necessário inferir todos os inteiros e doubles como strings.

cols = ['Unnamed: 0', 'app_id', 'app_name', 'review_id', 'language', 'timestamp_created', 'timestamp_updated', 'recommended',
        'votes_helpful', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free',
        'written_during_early_access', 'author.steamid', 'author.num_games_owned', 'author.num_reviews', 'author.playtime_forever',
        'author.playtime_last_two_weeks', 'author.playtime_at_review', 'author.last_played']

reviews = dd.read_csv('steam_reviews.csv',
                      usecols=cols,
                      sep=',',
                      encoding='UTF-8',
                      engine='python',
                      on_bad_lines='skip',
                      dtype={'Unnamed: 0': 'str',
                              'app_id': 'str',
                              'app_name': 'str',
                              'review_id': 'str',
                              'language': 'str',
                              'review': 'str',
                              'timestamp_created': 'str',
                              'timestamp_updated': 'str',
                              'recommended': 'str',
                              'votes_helpful': 'str',
                              'votes_funny': 'str',
                              'weighted_vote_score': 'str',
                              'comment_count': 'str',
                              'steam_purchase': 'str',
                              'received_for_free': 'str',
                              'written_during_early_access': 'str',
                              'author.steamid': 'str',
                              'author.num_games_owned': 'str',
                              'author.num_reviews': 'str',
                              'author.playtime_forever': 'str',
                              'author.playtime_last_two_weeks': 'str',
                              'author.playtime_at_review': 'str',
                              'author.last_played': 'str'})

In [5]:
reviews.head()

,Unnamed: 0,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,1611381629,1611381629,True,0,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1611343383.0
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,1611381030,1611381030,True,0,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1611386307.0
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,1611380800,1611380800,True,0,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1611383777.0
3,3,292030,The Witcher 3: Wild Hunt,85184605,english,1611379970,1611379970,True,0,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1611383744.0
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,1611379427,1611379427,True,0,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,1610788249.0


In [25]:
linhas = len(reviews)

print(f'O dataset possui {linhas} de linhas!')

O dataset possui 21756295 de linhas!


In [5]:
# Renomeando a coluna de índice

columns = reviews.columns.to_list()
columns[0] = "index"
cols_dict = {}

for idx, column in enumerate(reviews.columns.to_list()):
    cols_dict[column] = columns[idx]

reviews = reviews.rename(columns=cols_dict)

In [13]:
reviews.head()

,index,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,1611381629,1611381629,True,0,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1611343383.0
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,1611381030,1611381030,True,0,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1611386307.0
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,1611380800,1611380800,True,0,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1611383777.0
3,3,292030,The Witcher 3: Wild Hunt,85184605,english,1611379970,1611379970,True,0,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1611383744.0
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,1611379427,1611379427,True,0,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,1610788249.0


### Tratamento de dados

Devido ao fato do nosso dataset possuir uma coluna "reviews" que conta com a review de um usuário sobre um jogo e essa coluna possuir texto livre digitado diretamente pelo usuário, fica inviável trabalhar visto que essas reviews possuem vírgulas e por nosso arquivo .csv ser separado por vírgulas, isso acaba atrapalhando o nosso trabalho pois uma grande quantidade dos dados estão "sujos". 

Tentamos diversas opções para driblar esse problema, mas não chegamos em uma solução que nos permitisse continuar trabalhando com todos os dados. Diante disso, decidimos trabalhar com os reviews **APENAS** da língua chinesa, isso porque na língua chinesa não existe vírgula, o que não "suja" nossos dados, permitindo continuar o projeto.

In [6]:
chinese = reviews[reviews["language"] == "schinese"]
chinese.head()

,index,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,1611381629,1611381629,True,0,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1611343383.0
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,1611381030,1611381030,True,0,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1611386307.0
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,1611380800,1611380800,True,0,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1611383777.0
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,1611379427,1611379427,True,0,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,1610788249.0
8,8,292030,The Witcher 3: Wild Hunt,85183227,schinese,1611377703,1611377703,True,0,0,...,True,False,False,76561198130808993,581,17,6921.0,222.0,6921.0,1611317275.0


### Novo dataset

De qualquer forma, o novo dataset ainda é enorme, são 3.6 milhões de linhas.

In [30]:
linhas = len(chinese)

print(f"O novo dataset possui {linhas} linhas")

O novo dataset possui 3670537 linhas


### Análise exploratória

In [63]:
# Mudando para Pandas

df = chinese.compute()
type(df)

pandas.core.frame.DataFrame

In [64]:
# Mudando os tipos dos dados

convert = {
    "app_id": "int64",
    "review_id": "int64",
    "timestamp_created": "float64",
    "timestamp_updated": "float64",
    "recommended": "str",
    "votes_helpful": "int64",
    "votes_funny": "int64",
    "weighted_vote_score": "float64",
    "comment_count": "int64",
    "author.steamid": "int64",
    "author.playtime_forever": "float64",
    "author.playtime_at_review": "float64",
    "author.playtime_last_two_weeks": "float64",
    "author.num_reviews": "int64",
    "author.num_games_owned": "int64",
    "author.last_played": "float64",
    "received_for_free": "str",
    "steam_purchase": "str"
}

df = df.astype(convert)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3670537 entries, 0 to 131494
Data columns (total 22 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   index                           object 
 1   app_id                          int64  
 2   app_name                        object 
 3   review_id                       int64  
 4   language                        object 
 5   timestamp_created               float64
 6   timestamp_updated               float64
 7   recommended                     object 
 8   votes_helpful                   int64  
 9   votes_funny                     int64  
 10  weighted_vote_score             float64
 11  comment_count                   int64  
 12  steam_purchase                  object 
 13  received_for_free               object 
 14  written_during_early_access     object 
 15  author.steamid                  int64  
 16  author.num_games_owned          int64  
 17  author.num_reviews          

In [65]:
df["author.playtime_at_review"] = df["author.playtime_at_review"].apply(lambda x : x/60).round(1)
df["author.playtime_forever"] = df["author.playtime_forever"].apply(lambda x : x/60).round(1)
df["author.playtime_last_two_weeks"] = df["author.playtime_last_two_weeks"].apply(lambda x : x/60).round(1)
df.head()

,index,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,1.611382e+09,1.611382e+09,True,0,0,...,True,False,False,76561199095369542,6,2,31.8,24.1,31.8,1.611343e+09
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,1.611381e+09,1.611381e+09,True,0,0,...,True,False,False,76561198949504115,30,10,46.1,45.7,44.6,1.611386e+09
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,1.611381e+09,1.611381e+09,True,0,0,...,True,False,False,76561199090098988,5,1,17.7,17.7,17.7,1.611384e+09
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,1.611379e+09,1.611379e+09,True,0,0,...,True,False,False,76561199028326951,7,4,3.6,0.7,3.6,1.610788e+09
8,8,292030,The Witcher 3: Wild Hunt,85183227,schinese,1.611378e+09,1.611378e+09,True,0,0,...,True,False,False,76561198130808993,581,17,115.4,3.7,115.4,1.611317e+09


In [113]:
# Top 10 jogos com mais reviews

df_games = df["app_name"].value_counts().to_frame()
df_games = df_games.reset_index()
df_games.columns = ["app_name", "Qtde"]
df_games = df_games.sort_values("Qtde", ascending=False)
df_games.head(10)

,app_name,Qtde
0,PLAYERUNKNOWN'S BATTLEGROUNDS,836087
1,Grand Theft Auto V,242885
2,Tom Clancy's Rainbow Six Siege,136165
3,Wallpaper Engine,123648
4,The Witcher 3: Wild Hunt,101518
5,Hollow Knight,71856
6,Sekiro™: Shadows Die Twice,62848
7,DARK SOULS™ III,61758
8,Human: Fall Flat,61737
9,Terraria,60377


In [115]:
px.bar(df_games.head(15), x='app_name', y="Qtde")

In [116]:
# Top 10 usuários com mais reviews feitas

df_reviews = df["author.steamid"].value_counts().to_frame()
df_reviews = df_reviews.reset_index()
df_reviews.columns = ["id", "Qtde"]
df_reviews = df_reviews.sort_values("Qtde", ascending=False)
df_reviews.head(10)

,id,Qtde
0,76561198192166873,105
1,76561197989746995,82
2,76561198327150482,80
3,76561198854641697,66
4,76561198098930812,66
5,76561198176675135,62
6,76561198150474687,61
7,76561197998658899,60
8,76561198123990020,59
9,76561198147488050,59


In [43]:
# Quanto tempo de jogo para dar uma review?

df_timetorev = df.groupby("app_name", as_index=False).agg({"author.playtime_at_review": "median"}).sort_values("author.playtime_at_review", ascending=False).round(1)
df_timetorev.columns = ["app_name", "playtime_at_review (h)"]
df_timetorev.head(10)

,app_name,playtime_at_review (h)
182,PLAYERUNKNOWN'S BATTLEGROUNDS,201.3
273,Tom Clancy's Rainbow Six Siege,119.6
106,Football Manager 2019,102.8
280,Total War: WARHAMMER II,85.2
28,Black Desert Online,84.0
156,Monster Hunter: World,78.1
250,The Binding of Isaac: Rebirth,69.1
118,Grand Theft Auto V,68.9
127,Hearts of Iron IV,64.4
208,Rust,61.6


In [121]:
px.bar(df_timetorev.head(20), x='app_name', y='playtime_at_review (h)')

In [84]:
df_trues = df[df["recommended"] == "True"].groupby("app_name", as_index=False).agg({"recommended": "count"})
df_trues.columns = ["app_name", "trues"]
df_trues.head(10)

,app_name,trues
0,20XX,166
1,A Hat in Time,1589
2,A Short Hike,489
3,A Way Out,842
4,ARK: Survival Evolved,32885
5,ATLAS,840
6,Age of Empires II (2013),5417
7,Age of Empires: Definitive Edition,429
8,American Truck Simulator,2391
9,Among Us,3498


In [83]:
df_worst = df[["app_name", "recommended"]].groupby("app_name", as_index=False).agg({"recommended": "count"})
df_worst.columns = ["app_name", "total"]
df_worst.head(10)

,app_name,total
0,20XX,201
1,A Hat in Time,1645
2,A Short Hike,492
3,A Way Out,1136
4,ARK: Survival Evolved,43304
5,ATLAS,4404
6,Age of Empires II (2013),5738
7,Age of Empires: Definitive Edition,619
8,American Truck Simulator,2559
9,Among Us,3997


In [87]:
df_worst = df_worst.set_index("app_name")
df_trues = df_trues.set_index("app_name")

In [90]:
df_rate = df_worst.join(df_trues, on="app_name", how="inner")
df_rate = df_rate.reset_index()
df_rate.head(10)

,app_name,total,trues
0,20XX,201,166
1,A Hat in Time,1645,1589
2,A Short Hike,492,489
3,A Way Out,1136,842
4,ARK: Survival Evolved,43304,32885
5,ATLAS,4404,840
6,Age of Empires II (2013),5738,5417
7,Age of Empires: Definitive Edition,619,429
8,American Truck Simulator,2559,2391
9,Among Us,3997,3498


In [125]:
# Piores jogos

df_rate["rate"] = (((df_rate["total"]-df_rate["trues"])/df_rate["total"])*100).round(2)
df_rate = df_rate.sort_values("rate", ascending=False).head(15)
df_rate.head(20)

,app_name,total,trues,rate
31,Bless Online,39,7,82.05
206,RollerCoaster Tycoon World,148,27,81.76
5,ATLAS,4404,840,80.93
290,Umbrella Corps,463,91,80.35
161,NBA 2K18,11276,2524,77.62
163,NBA 2K21,8507,2028,76.16
61,DYNASTY WARRIORS 9,5285,1296,75.48
212,SOS,3373,898,73.38
82,Down To One,56,15,73.21
162,NBA 2K19,12715,3485,72.59


In [131]:
px.bar(df_rate.head(20), x='app_name', y='rate', color='app_name')

In [104]:
# Qual o jogo que é mais dado de presente do que comprado?

df_gift_total = df.groupby("app_name", as_index=False).agg({"received_for_free": "count"})
df_gift_total.columns = ["app_name", "total"]
df_gift_total.head()

,app_name,total
0,20XX,201
1,A Hat in Time,1645
2,A Short Hike,492
3,A Way Out,1136
4,ARK: Survival Evolved,43304


In [103]:
df_gift_trues = df[df["received_for_free"] == "True"].groupby("app_name", as_index=False).agg({"received_for_free": "count"})
df_gift_trues.columns = ["app_name", "trues"]
df_gift_trues.head()

,app_name,trues
0,20XX,1
1,A Hat in Time,22
2,A Short Hike,3
3,A Way Out,12
4,ARK: Survival Evolved,1028


In [105]:
df_gift_total = df_gift_total.set_index("app_name")
df_gift_trues = df_gift_trues.set_index("app_name")

In [106]:
df_gifts = df_gift_total.join(df_gift_trues, on="app_name", how="inner")
df_gifts = df_gifts.reset_index()

In [132]:
df_gifts["rate"] = ((df_gifts["trues"]/df_gifts["total"])*100).round(2)
df_gifts = df_gifts.sort_values("rate", ascending=False).head(15)
df_gifts.head(20)

,app_name,total,trues,rate
115,Guacamelee! Super Turbo Championship Edition,401,146,36.41
266,Totally Accurate Battlegrounds,3754,1034,27.54
189,Rapture Rejects,168,35,20.83
258,Tomb Raider,37596,6907,18.37
157,Neon Hardcorps,31,5,16.13
103,For Honor,5649,634,11.22
114,Grimm's Hollow,58,6,10.34
16,BATTALION 1944,136,14,10.29
74,Doki Doki Literature Club,13497,1309,9.70
147,Mirror,41810,4052,9.69


In [133]:
px.bar(df_gifts, x='app_name', y='rate', color='app_name')

2022-11-29 23:54:23,351 - distributed.nanny - WARNING - Restarting worker
2022-11-29 23:54:23,360 - distributed.nanny - WARNING - Restarting worker
2022-11-29 23:54:23,402 - distributed.nanny - WARNING - Restarting worker
2022-11-29 23:54:23,411 - distributed.nanny - WARNING - Restarting worker


### Prevendo a positividade da review usando Machine Learning

Vamos criar um modelo de Regressão Linear para prever se uma review é positiva ou não (recommended) usando a biblioteca do scikit-learn.

In [38]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [27]:
# Analisando os tipos dos nossos dados

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3670537 entries, 0 to 131494
Data columns (total 22 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   index                           object
 1   app_id                          object
 2   app_name                        object
 3   review_id                       object
 4   language                        object
 5   timestamp_created               object
 6   timestamp_updated               object
 7   recommended                     object
 8   votes_helpful                   object
 9   votes_funny                     object
 10  weighted_vote_score             object
 11  comment_count                   object
 12  steam_purchase                  object
 13  received_for_free               object
 14  written_during_early_access     object
 15  author.steamid                  object
 16  author.num_games_owned          object
 17  author.num_reviews              object
 18  aut

In [29]:
# Mudando a variável que desejamos prever de booleano para inteiro para o modelo preditivo funcionar

df["recommended"].replace(["True", "False"], [1, 0], inplace=True)
df["steam_purchase"].replace(["True", "False"], [1, 0], inplace=True)
df["received_for_free"].replace(["True", "False"], [1, 0], inplace=True)
df.head(5)

,index,app_id,app_name,review_id,language,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,1611381629,1611381629,1,0,0,...,1,0,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1611343383.0
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,1611381030,1611381030,1,0,0,...,1,0,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1611386307.0
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,1611380800,1611380800,1,0,0,...,1,0,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1611383777.0
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,1611379427,1611379427,1,0,0,...,1,0,False,76561199028326951,7,4,217.0,42.0,217.0,1610788249.0
8,8,292030,The Witcher 3: Wild Hunt,85183227,schinese,1611377703,1611377703,1,0,0,...,1,0,False,76561198130808993,581,17,6921.0,222.0,6921.0,1611317275.0


In [36]:
# Construindo o modelo

x = df[["steam_purchase", "received_for_free", "author.playtime_forever", "author.playtime_at_review"]].fillna(0)
y = df["recommended"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)

In [37]:
reg = LogisticRegression()
reg.fit(x_train, y_train)

LogisticRegression()

In [39]:
y_pred = reg.predict(x_test)

In [41]:
# Acurácia e precisão do modelo

print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred, zero_division=1))

Accuracy:  0.7680640995602827
Precision:  0.7757971724380691


### Tentando melhorar o modelo

Apesar da acurácia e precisão do modelos estarem perto de 75%, que é um bom número, podemos tentar melhorá-lo utilizando algumas técnicas, a que usaremos hoje será a de **Feature selection** do scikit learn. Essa técnica nos indica quais são as melhores variáveis para colocar no modelo. Isso pode aumentar a eficácia e precisão do modelo visto que na primeira iteração as variáveis foram escolhidas a olho.

In [53]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [50]:
x = df.select_dtypes(exclude=[object]).fillna(0)
y = df["recommended"]

In [51]:
best = SelectKBest(score_func=chi2, k=3)
fit = best.fit(x, y)

In [55]:
df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(x.columns)

features_scores = pd.concat([df_columns, df_scores], axis=1)
features_scores.columns = ['Features', 'Score']
features_scores.sort_values(by='Score', ascending=False)

,Features,Score
12,author.num_games_owned,1.316312e+12
1,review_id,2.632189e+11
2,timestamp_created,4.627654e+10
3,timestamp_updated,4.294046e+10
17,author.last_played,1.498652e+10
0,app_id,8.621650e+09
14,author.playtime_forever,8.065076e+09
16,author.playtime_at_review,6.818464e+09
6,votes_funny,3.385262e+09
15,author.playtime_last_two_weeks,1.881542e+06


Como nosso dataframe possui dados que não impactam diretamente na positividade da review, o DataFrame ficou meio estranho, mas é possível visualizar algumas features que não estavam antes no modelo e podem ser adicionadas, e outras que podem ser excluídas. 

In [61]:
# Removendo features indesejadas e adicionando novas

x = df[["author.playtime_forever", "author.playtime_at_review", "author.num_games_owned"]].fillna(0)
y = df["recommended"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)

In [62]:
reg = LogisticRegression()
reg.fit(x_train, y_train)

LogisticRegression()

In [63]:
y_pred = reg.predict(x_test)

In [64]:
# Acurácia e precisão do novo modelo

print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
print("Precision: ", metrics.precision_score(y_test, y_pred, zero_division=1))

Accuracy:  0.7734847370323349
Precision:  0.7814873258414702


### Conclusão da análise de Machine Learning

Como foi possível perceber, ambos os modelos não fujiram muita da casa dos 75-80%, o que não é ruim, mas com certeza podia melhorar. Vale lembrar que a base de dados não foi coletada explicitamente para Machine Learning, portanto, faltam dados importantes para construir um modelo muito preciso e eficaz. No entanto, o resultado final contenta, visto que não tiveram barreiras no dataset que impossibilitariam a gente de sequer realizar a análise de Machine Learning.

Obviamente que poderíamos ter tentado outras técnicas mais avançadas de Machine Learning, mas esse nunca foi o propósito do projeto.

# Conclusão Final

Como grupo, adoramos fazer esse projeto principalmente pelo tema escolhido, algo que nos identificamos bastante. Ao longo da disciplina, fomos criando um gosto muito grande por ela, e recomendamos diversas vezes para colegas que ainda não cursaram. Aprendemos coisas importantes e muitas coisas novas, mesmo que algumas tecnologias já foram mostradas pra gente, coisas novas surgiram das aulas de Big Data, o que foi incrível. Ficamos muito felizes com o resultado da disciplina, pois é com certeza um tema que vai crescer bastante ainda e não só na área de engenharia/tecnologia como muitos imaginam, vamos usar o que aprendemos no futuro com certeza, vai ser um diferencial.

Falando sobre o projeto, muitas das nossas análises fizeram enorme sentido do ponto de vista de uma pessoa que já conhece esse ecossistema de games que é a Steam. Porém, ainda sim foi possível descobrir coisas que pra gente como usuário era impossível de visualizar apenas usando a Steam de forma padrão, nós precisávamos dos dados! Fizemos descobertas sobre jogos em si, sobre usuários, sobre quantidade de horas jogadas e sobre jogos recebidos de presente. A verdade é que para uma pessoa de fora da comunidade de jogos, as informações podem não fazer sentido, mas para a gente fez, e muito. É gratificante estudar sobre coisas que gostamos e que consumimos durante o dia a dia. Tenho certeza que as grandes empresas se já não fazem, devem fazer estudos parecidos com o disponibilizado nesse projeto, pois é muita informação para absorver e trabalhar em cima. Nas análises é possível observar grandes nomes da indústria como por exemplo Grand Theft Auto V, PLAYERUNKNOWN'S BATTLEGROUNDS, The Witcher 3, Rainbow Six Siege e muitos outros. Também é interessante analisar a enorme diferença nos dados, os usuários são muito diferentes, nós nem imaginávamos que seria assim, mas a Steam é uma plataforma enorme que abriga jogadores do mundo todo. 

A limpeza dos dados foi chata, muito por conta de uma coluna ser texto livre, o que acarreta em muitas vírgulas e que por sua vez atrapalha a leitura dos dados de um arquivo CSV separado por vírgula como no nosso caso. Entretanto, achamos um caminho que continuava na linha de pensamento, visto que a região do mundo não mudaria a qualidade dos dados. Nas análises, mostramos informações como, jogo mais popular, usuários com mais reviews (números absurdos por sinal), a média de tempo de jogo para uma review ser feita em cada jogo (valor aproximado, mas da pra ter uma ideia interessante), quais são os jogos com a maior porcentagem de reviews negativas (números assustadores mesmo com quantidade de reviews altas) e quais são os jogos com a maior porcentagem de reviews feitas a partir de um presente (análise super interessante que não passa pela cabeça das pessoas o quanto isso pode afetar um jogo). Sem falar no nosso modelo preditivo com Machine Learning para prever positividade de reviews.